In [1]:
import os
import re
import pandas as pd
import numpy as np
import qiime2
from qiime2 import Artifact
from qiime2.plugins import metadata, feature_table, diversity, emperor
from q2_emperor import plot, procrustes_plot, biplot, generic_plot
from scipy.spatial.distance import squareform, pdist
import skbio

In [2]:
!ls ..

Dockerfile  Makefile  README.md  api  notebooks  run_server.sh


In [4]:
# Change directory to import function from api
os.chdir('..')

In [5]:
from api.utils import qiime2PCoA
from api.gnps import Proteosafe

In [6]:
taskid = 'a0b4377f6e4540268602c30fedf69b0b'
# Can you see the available options in the code? We have to create a 
# drop-down menu for the user to choose
workflow = 'FBMN'
metric = 'euclidean'
gnps_result = Proteosafe(taskid, workflow)
gnps_result.get_gnps()

if not os.path.exists(f'notebooks/{taskid}'):
    os.mkdir(f'notebooks/{taskid}')

pcoa_obj = qiime2PCoA(gnps_result.meta, gnps_result.feat,
                      out_dir=f'notebooks/{taskid}',
                      metric=metric)
pcoa_file = f'notebooks/{taskid}/{taskid}.qzv'
pcoa_obj.visualization.save(pcoa_file)

/ClusterApp/api/utils.py:17: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_metadata.columns = sample_metadata.columns.str.replace('\s', '_')


'notebooks/a0b4377f6e4540268602c30fedf69b0b/a0b4377f6e4540268602c30fedf69b0b.qzv'

In [7]:
!ls notebooks/a0b4377f6e4540268602c30fedf69b0b

a0b4377f6e4540268602c30fedf69b0b.qzv


In [8]:
# open the html file on your computer
# copy it if necessary
!firefox a0b4377f6e4540268602c30fedf69b0b/index.html

/bin/sh: 1: firefox: not found
